In [1]:
from training.training import finetune
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer
from filtering import compute_perplexity, compute_solution_perplexity, filter_dataset
import numpy as np

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_

In [2]:
code_exercises = load_dataset("jinaai/code_exercises")["train"].select(range(1000)) # choose 1k samples
train_test_split = code_exercises.train_test_split(test_size=0.1)
train_eval_split = train_test_split["train"].train_test_split(test_size=1/9)

In [3]:
model_name = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.__dict__.get("pad_token") is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [4]:
raw_perplexity_dataset = train_test_split["test"].map(compute_solution_perplexity, fn_kwargs={"model": model, "tokenizer": tokenizer}, batched=True, batch_size=8)
raw_perplexity = np.mean(raw_perplexity_dataset["solution_perplexity"])
print("Raw perplexity:", raw_perplexity)

Map: 100%|██████████| 100/100 [00:06<00:00, 15.63 examples/s]

Raw perplexity: 340172.42172851565


In [5]:
finetuned_model, eval_loss = finetune(model, tokenizer, train_eval_split)

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.258700,0.144275
2,0.080000,0.156309


In [6]:
finetuned_perplexity_dataset = train_test_split["test"].map(compute_solution_perplexity, fn_kwargs={"model": finetuned_model, "tokenizer": tokenizer}, batched=True, batch_size=8)
finetuned_perplexity = np.mean(finetuned_perplexity_dataset["solution_perplexity"])
print("Test perplexity for model finetuned on the whole train dataset", finetuned_perplexity)

Map: 100%|██████████| 100/100 [00:06<00:00, 15.97 examples/s]

Test perplexity for model finetuned on the whole train dataset 27171.49729288578


In [7]:
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
train_dataset = train_eval_split["train"].map(compute_solution_perplexity, fn_kwargs={"model": model, "tokenizer": tokenizer}, batched=True, batch_size=8)
eval_dataset = train_eval_split["test"]

perplexities = []
for part in ["low", "mid", "high"]:
    model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
    dataset = DatasetDict({
        "train": filter_dataset(train_dataset, "solution_perplexity", 0.5, part),
        "test": eval_dataset
    })
    finetuned_model, eval_loss = finetune(model, tokenizer, dataset)

    finetuned_perplexity_dataset = train_test_split["test"].map(compute_solution_perplexity, fn_kwargs={"model": finetuned_model, "tokenizer": tokenizer}, batched=True, batch_size=8)
    perplexities.append(np.mean(finetuned_perplexity_dataset["solution_perplexity"]))

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Epoch,Training Loss,Validation Loss
1,0.314200,0.147809
2,0.056600,0.164914


Map: 100%|██████████| 100/100 [00:06<00:00, 15.95 examples/s]
Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a

Epoch,Training Loss,Validation Loss
1,0.365800,0.147414
2,0.079900,0.158094


Map: 100%|██████████| 100/100 [00:06<00:00, 15.95 examples/s]
Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a

Epoch,Training Loss,Validation Loss
1,0.387500,0.143763
2,0.101000,0.158471


Map: 100%|██████████| 100/100 [00:06<00:00, 15.96 examples/s]


In [8]:
for ppl, part in zip(perplexities, ["low", "middle", "high"]):
    print(f"Test perplexity for model finetuned on {part} perplexity samples:", ppl)

Test perplexity for model finetuned on low perplexity samples: 92641.61225682497
Test perplexity for model finetuned on middle perplexity samples: 72353.55313482403
Test perplexity for model finetuned on high perplexity samples: 65712.25809334636
